In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# To scale the data using z-score 
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# Algorithms to use
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC, SVR

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import RandomForestClassifier

# Metrics to evaluate the model
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve,recall_score, mean_squared_error
from sklearn import tree


# For tuning the model
from sklearn.model_selection import GridSearchCV

In [13]:
occupancy_data_2023 = pd.read_csv("./data/occupancy/Daily_shelter_overnight_occupancy.csv", low_memory=False)
occupancy_data_2021 = pd.read_csv("./data/occupancy/daily-shelter-overnight-service-occupancy-capacity-2021.csv", low_memory=False)
occupancy_data_2022 = pd.read_csv("./data/occupancy/daily-shelter-overnight-service-occupancy-capacity-2022.csv", low_memory=False)

occupancy_data_2022['OCCUPANCY_DATE'] = pd.to_datetime(occupancy_data_2022['OCCUPANCY_DATE'], format='%y-%m-%d').dt.strftime('%Y-%m-%d')
occupancy_data_2021['OCCUPANCY_DATE'] = pd.to_datetime(occupancy_data_2021['OCCUPANCY_DATE'], format='%y-%m-%d').dt.strftime('%Y-%m-%d')

In [14]:
all_shelter_data = pd.concat([occupancy_data_2023, occupancy_data_2022, occupancy_data_2021])

In [15]:
all_shelter_data['OCCUPANCY_DATE'] = pd.to_datetime(all_shelter_data['OCCUPANCY_DATE'], format='%Y-%m-%d')

In [16]:
toronto_data = all_shelter_data[all_shelter_data["LOCATION_CITY"]=="Toronto"]

In [17]:
toronto_data['MONTH'] = toronto_data['OCCUPANCY_DATE'].dt.month
toronto_data['DAY'] = toronto_data['OCCUPANCY_DATE'].dt.day
toronto_data['YEAR'] = toronto_data['OCCUPANCY_DATE'].dt.year

<ipython-input-17-dee387474e21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toronto_data['MONTH'] = toronto_data['OCCUPANCY_DATE'].dt.month
<ipython-input-17-dee387474e21>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toronto_data['DAY'] = toronto_data['OCCUPANCY_DATE'].dt.day
<ipython-input-17-dee387474e21>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [1]:
final_df.info()

NameError: name 'final_df' is not defined

In [22]:
toronto_data_dr = toronto_data.drop(columns = ['_id', 'OCCUPANCY_DATE', 'ORGANIZATION_ID', 'ORGANIZATION_NAME', 'SHELTER_ID', 'SHELTER_GROUP',
                            'LOCATION_ID', 'LOCATION_NAME', 'LOCATION_ADDRESS', 'LOCATION_CITY', 'LOCATION_PROVINCE', 'PROGRAM_ID', 
                            'PROGRAM_NAME', 'OVERNIGHT_SERVICE_TYPE', 'PROGRAM_AREA', 'SERVICE_USER_COUNT', 'CAPACITY_FUNDING_BED',
                            'OCCUPIED_BEDS', 'UNOCCUPIED_BEDS', 'CAPACITY_FUNDING_ROOM', 'OCCUPIED_ROOMS', 'UNOCCUPIED_ROOMS',
                            ])

In [28]:
toronto_data_dr_nan = toronto_data_dr[toronto_data_dr["PROGRAM_MODEL"].notna()]

In [31]:
# Creating list of dummy columns
to_get_dummies_for = ['SECTOR']

# Creating dummy variables
toronto_data_dr_nan = pd.get_dummies(data = toronto_data_dr_nan, columns = to_get_dummies_for)      

# Mapping overtime and attrition
dict_prog_mod = {'Emergency': 1, 'Transitional':0}
dict_cap_type = {'Bed Based Capacity': 1, 'Room Based Capacity': 0}


toronto_data_dr_nan['prog_mod'] = toronto_data_dr_nan["PROGRAM_MODEL"].map(dict_prog_mod)
toronto_data_dr_nan['cap_type'] = toronto_data_dr_nan["CAPACITY_TYPE"].map(dict_cap_type)

In [37]:
toronto_data_dr_nan = toronto_data_dr_nan.drop(columns = ["PROGRAM_MODEL", "CAPACITY_TYPE"])

In [48]:
toronto_data_dr_nan["postal_code"] = toronto_data_dr_nan['LOCATION_POSTAL_CODE'].apply(lambda x: x.split(" ")[0])

In [51]:
toronto_data_dr_nan["postal_code"].unique()

array(['M9W', 'M5S', 'M6R', 'M6H', 'M6G', 'M1P', 'M3M', 'M4P', 'M1E',
       'M5V', 'M1H', 'M5A', 'M1M', 'M6N', 'M5C', 'M5T', 'M5H', 'M6C',
       'M5B', 'M4C', 'M5J', 'M5E', 'M3B', 'M4X', 'M4Y', 'M6A', 'M6K',
       'M1K', 'M1R', 'M2H', 'M6E', 'M4K', 'M4W', 'M4M', 'M8Y', 'M2M',
       'M6P', 'M9V', 'M5R', 'M4T', 'M2N', 'M4L', 'M6J', 'M3N'],
      dtype=object)

In [53]:
toronto_data_dr_nan["postal_code"] = toronto_data_dr_nan["postal_code"].astype('category')

In [56]:
toronto_data_dr_nan["postal_code_num"] = pd.Categorical(toronto_data_dr_nan["postal_code"]).codes

In [58]:
final_df = toronto_data_dr_nan.drop(columns = ["postal_code", "LOCATION_POSTAL_CODE"])

In [62]:
final_df = final_df.fillna(value=0.0)

In [96]:
final_df["OCCUPANCY_RATE_ROOMS"] = final_df["OCCUPANCY_RATE_ROOMS"]/100.0

In [97]:
final_df["OCCUPANCY_RATE_BEDS"] = final_df["OCCUPANCY_RATE_BEDS"]/100.0

# Room predictor

In [122]:
Y = final_df["OCCUPANCY_RATE_ROOMS"]
X = final_df.drop(columns = ['OCCUPANCY_RATE_ROOMS', "OCCUPANCY_RATE_BEDS"])

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)

In [124]:
sc=StandardScaler()

# Fit_transform on train data
X_train_scaled=sc.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled, columns=X.columns)

# Transform on test data
X_test_scaled=sc.transform(X_test)
X_test_scaled=pd.DataFrame(X_test_scaled, columns=X.columns)

In [125]:
svm_rbf=SVR(kernel='rbf', max_iter = 10000)
# Fit the model
svm_rbf.fit(X_train_scaled,y_train)

SVR(max_iter=10000)

In [126]:
print(f"Training score: {svm_rbf.score(X_train_scaled, y_train)}")
print(f"Testing score: {svm_rbf.score(X_test_scaled, y_test)}")

Training score: 0.9863632175344624
Testing score: 0.9861598809310522


In [127]:
train_pred = svm_rbf.predict(X_train_scaled)
test_pred = svm_rbf.predict(X_test_scaled)
print(f"Training mse: {mean_squared_error(y_train, train_pred, squared=False)}")
print(f"Testing mse: {mean_squared_error(y_test, test_pred, squared=False)}")

Training mse: 0.05013722337267246
Testing mse: 0.05027222938313009


# Beds Predictor

In [128]:
Y = final_df["OCCUPANCY_RATE_BEDS"]
X = final_df.drop(columns = ['OCCUPANCY_RATE_ROOMS', "OCCUPANCY_RATE_BEDS"])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)

sc=StandardScaler()

# Fit_transform on train data
X_train_scaled=sc.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled, columns=X.columns)

# Transform on test data
X_test_scaled=sc.transform(X_test)
X_test_scaled=pd.DataFrame(X_test_scaled, columns=X.columns)

In [129]:
svm_rbf2=SVR(kernel='rbf', max_iter = 10000)
# Fit the model
svm_rbf2.fit(X_train_scaled,y_train)

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


SVR(max_iter=10000)

In [130]:
print(f"Training score: {svm_rbf2.score(X_train_scaled, y_train)}")
print(f"Testing score: {svm_rbf2.score(X_test_scaled, y_test)}")

Training score: 0.9615725930996076
Testing score: 0.9608583254648059


In [131]:
train_pred = svm_rbf2.predict(X_train_scaled)
test_pred = svm_rbf2.predict(X_test_scaled)
print(f"Training mse: {mean_squared_error(y_train, train_pred, squared=False)}")
print(f"Testing mse: {mean_squared_error(y_test, test_pred, squared=False)}")

Training mse: 0.08403209511132166
Testing mse: 0.08448791811351003


### Only Room data as input

In [111]:
Y = final_df[final_df["cap_type"]==0]["OCCUPANCY_RATE_ROOMS"]
X = final_df[final_df["cap_type"]==0].drop(columns = ['OCCUPANCY_RATE_ROOMS', "OCCUPANCY_RATE_BEDS", 'cap_type'])

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)
sc=StandardScaler()

# Fit_transform on train data
X_train_scaled=sc.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled, columns=X.columns)

# Transform on test data
X_test_scaled=sc.transform(X_test)
X_test_scaled=pd.DataFrame(X_test_scaled, columns=X.columns)

In [113]:
svm_rbf_room=SVR(kernel='rbf', max_iter = 100000)
# Fit the model
svm_rbf_room.fit(X_train_scaled,y_train)
print(f"Training score: {svm_rbf_room.score(X_train_scaled, y_train)}")
print(f"Testing score: {svm_rbf_room.score(X_test_scaled, y_test)}")

Training score: 0.09925118461509874
Testing score: 0.10878320245185169


In [114]:
train_pred = svm_rbf_room.predict(X_train_scaled)
test_pred = svm_rbf_room.predict(X_test_scaled)
print(f"Training mse: {mean_squared_error(y_train, train_pred, squared=False)}")
print(f"Testing mse: {mean_squared_error(y_test, test_pred, squared=False)}")

Training mse: 0.06856938833810024
Testing mse: 0.07086391114121894


### Only Bed data as input

In [116]:
Y = final_df[final_df["cap_type"]==1]["OCCUPANCY_RATE_BEDS"]
X = final_df[final_df["cap_type"]==1].drop(columns = ['OCCUPANCY_RATE_ROOMS', "OCCUPANCY_RATE_BEDS", 'cap_type'])

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)
sc=StandardScaler()

# Fit_transform on train data
X_train_scaled=sc.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled, columns=X.columns)

# Transform on test data
X_test_scaled=sc.transform(X_test)
X_test_scaled=pd.DataFrame(X_test_scaled, columns=X.columns)

In [118]:
svm_rbf_bed=SVR(kernel='rbf', max_iter = 100000)
# Fit the model
svm_rbf_bed.fit(X_train_scaled,y_train)
print(f"Training score: {svm_rbf_bed.score(X_train_scaled, y_train)}")
print(f"Testing score: {svm_rbf_bed.score(X_test_scaled, y_test)}")

Training score: 0.17634674125120953
Testing score: 0.1559900607905912


In [119]:
train_pred = svm_rbf_bed.predict(X_train_scaled)
test_pred = svm_rbf_bed.predict(X_test_scaled)
print(f"Training mse: {mean_squared_error(y_train, train_pred, squared=False)}")
print(f"Testing mse: {mean_squared_error(y_test, test_pred, squared=False)}")

Training mse: 0.09144841489829611
Testing mse: 0.09410094024346789
